# HF 二阶核坐标梯度性质

上一节我们我们回顾了 HF 方法下的一阶核坐标梯度性质．现在我们考虑二阶核坐标梯度，即得到核坐标 Hessian 矩阵．事实上，我们已经在之前的文档中求得了电性质的二阶梯度，即极化率．尽管同为二阶梯度性质，但核坐标梯度相比于电性质梯度的公式更为复杂，这已经在上一篇文档中有所体现．我们仍然依照 Yamaguchi 书的思路写出公式．

In [ ]:
from pyscf import gto, scf, grad, hessian, data, lib
import numpy as np
np.set_printoptions(5, linewidth=150, suppress=True)

## 准备工作

### 顶层函数计算 HF 梯度

由于核坐标二阶梯度的公式与张量相对复杂，因此这里不再使用水分子计算，而使用过氧化氢分子计算．这样就可以从维度上直接判断哪些变量与原子相关，哪些则与三维空间本身有关．

In [ ]:
mol = gto.Mole()
mol.atom = """
O  0.0  0.0  0.0
O  0.0  0.0  1.5
H  1.0  0.0  0.0
H  0.0  1.0  1.5
"""
mol.basis = "6-31G"
mol.build()

In [ ]:
scf_eng = scf.RHF(mol)
energy_RHF = scf_eng.kernel()

In [ ]:
scf_grad = grad.RHF(scf_eng)
grad_RHF = scf_grad.kernel()

In [ ]:
scf_hess = hessian.RHF(scf_eng)
hess_RHF = scf_hess.kernel()
hess_RHF.shape

### 与 Gaussian 结果进行比对

即使只有 4 个原子，Hessian 矩阵的维度也已经到 144 维了 (在 Cartesian 而非 Normal Mode 坐标下)．比对数据就会比较困难．在这里，我们可以试着用 Gaussian 所输出的 [fchk 文件](include/HF-hess.fch) (对应的 [输入卡](include/HF-hess.gjf)、[输出文件](include/HF-hess.out)) 所给出的结果，辅以下面的脚本，进行核验．

In [ ]:
def val_from_fchk(key, file_path):
    flag_read = False; expect_size = -1; vec = []
    with open(file_path, "r") as file:
        for l in file:
            if (l[:len(key)] == key):
                try: expect_size = int(l[len(key):].split()[2]); flag_read = True; continue
                except IndexError: return float(l[len(key):].split()[1])
            if (flag_read):
                try: vec += [ float(i) for i in l.split() ]
                except ValueError: break
    if len(vec) != expect_size: raise ValueError("Number of expected size is not consistent with read-in size!")
    return np.array(vec)

In [ ]:
energy_Gaussian = val_from_fchk("SCF Energy", "include/HF-hess.fch")
grad_Gaussian = val_from_fchk("Cartesian Gradient", "include/HF-hess.fch")
hess_Gaussian = val_from_fchk("Cartesian Force Constants", "include/HF-hess.fch")

能量、梯度信息都能立即核验，但 Gaussian 中输出的 Hessian 矩阵是压平后的下三角矩阵．因此，我们也需要稍作处理后才能给出核验结果．

In [ ]:
print(np.allclose(energy_RHF, energy_Gaussian))
print(np.allclose(grad_RHF.reshape(-1), grad_Gaussian))
d_hess = mol.natm * 3
print(np.allclose(hess_RHF.swapaxes(1, 2).reshape(d_hess, d_hess)[np.tril_indices(d_hess)], hess_Gaussian))

### HF 重要中间矩阵

In [ ]:
nao = mol.nao
nmo = scf_eng.mo_energy.shape[0]
nelec = mol.nelectron
nocc = mol.nelec[0]
nvir = nmo - nocc

C = scf_eng.mo_coeff
Co = C[:, :nocc]
Cv = C[:, nocc:]
e = scf_eng.mo_energy
eo = e[:nocc]
ev = e[nocc:]
mo_occ = scf_eng.mo_occ

D = scf_eng.make_rdm1()
De = np.einsum("ui, i, vi -> uv", C, e * mo_occ, C)

### 梯度重要量

在梯度中，我们经常需要定义新的矩阵，而其维度通常与原子数有关．原子数定义如下：

In [ ]:
natm = mol.natm

一些单电子积分的定义也放在这里．对单电子积分的初始化不仅是为了方便，同时还通过积分在内存中的储存，避免在后续的代码中多次计算这些积分．但不少积分仍然需要现算，譬如需要进行坐标重新定义的众多 $1 / |\boldsymbol{r}|$ 的积分．下面的记号 $T$ 一次代表 $ts$，即该角标对应的维度对于任何分子与基组下，均是 9 维．

Gradient 贡献张量

* `int1e_ipovlp`, $t \mu \nu$: $\langle \partial_t \mu | \nu \rangle$

* `int1e_ipkin`, $t \mu \nu$: $\langle \partial_t \mu | \hat t | \nu \rangle$

* `int1e_ipnuc`, $t \mu \nu$: $\langle \partial_t \mu | \hat v_\mathrm{nuc} | \nu \rangle$

* `int2e_ip1`, $t \mu \nu \kappa \tau$: $\langle \partial_t \mu \nu | \kappa \lambda \rangle$

Hessian 贡献张量

* `int1e_ipipkin`, $T \mu \nu$ : $\langle \partial_t \partial_s \mu | \hat t | \nu \rangle$

* `int1e_ipkinip`, $T \mu \nu$ : $\langle \partial_t \mu | \hat t | \partial_s \nu \rangle$

* `int1e_ipipnuc`, $T \mu \nu$ : $\langle \partial_t \partial_s \mu | \hat v_\mathrm{nuc} | \nu \rangle$

* `int1e_ipnucip`, $T \mu \nu$ : $\langle \partial_t \mu | \hat v_\mathrm{nuc} | \partial_s \nu \rangle$

* `int2e_ipip1`, $T \mu \nu \kappa \tau$ : $(\partial_{r_t} \partial_{r_s} \mu \nu | \kappa \lambda)$

* `int2e_ipvip1`, $T \mu \nu \kappa \tau$ : $(\partial_{r_t} \mu \partial_{r_s} \nu | \kappa \lambda)$

* `int2e_ip1ip2`, $T \mu \nu \kappa \tau$ : $(\partial_{r_t} \mu \nu | \partial_{r_s} \kappa \lambda)$

* `int1e_ipipovlp`, $T \mu \nu$ : $(\partial_{r_t} \partial_{r_s} \mu | \nu)$

* `int1e_ipovlpip`, $T \mu \nu$ : $(\partial_{r_t} \mu | \partial_{r_s} \nu)$

In [ ]:
# grad-contrib
int1e_ipovlp = mol.intor('int1e_ipovlp')
int1e_ipkin = mol.intor("int1e_ipkin")
int1e_ipnuc = mol.intor("int1e_ipnuc")
int2e_ip1 = mol.intor("int2e_ip1")

In [ ]:
# hess-contrib
int1e_ipipkin = mol.intor("int1e_ipipkin")
int1e_ipkinip = mol.intor("int1e_ipkinip")
int1e_ipipnuc = mol.intor("int1e_ipipnuc")
int1e_ipnucip = mol.intor("int1e_ipnucip")
int2e_ipip1 = mol.intor("int2e_ipip1")
int2e_ipvip1 = mol.intor("int2e_ipvip1")
int2e_ip1ip2 = mol.intor("int2e_ip1ip2")
int1e_ipipovlp = mol.intor("int1e_ipipovlp")
int1e_ipovlpip = mol.intor("int1e_ipovlpip")

### 便利函数

在二阶梯度中，我们会非常经常地碰到抽提矩阵或张量中，一部分原子的贡献．为了拿出这一部分原子的角标，我们可以使用下述函数调取角标．

In [ ]:
def mol_slice(atm_id):
    _, _, p0, p1 = mol.aoslice_by_atom()[atm_id]
    return slice(p0, p1)

## HF 电子能量二阶梯度公式回顾

### 二阶梯度总公式

根据 Yamaguchi 二阶梯度公式 (V.2)，我们可以将电子态能量的二阶梯度写为以下自定义的记号：

\begin{align}
\frac{\partial^2 E_\mathrm{elec}}{\partial A_t \partial B_s} 
&= 2 h_{ii}^{A_t B_s} + 2 (ii | jj)^{A_t B_s} - (ij | ij)^{A_t B_s} - 2 S_{ii}^{A_t B_s} \varepsilon_i - 2 \eta_{ii}^{A_t B_s} \varepsilon_i \\
&\quad + 4 U_{pi}^{B_s} F_{pi}^{A_t} + 4 U_{pi}^{A_t} F_{pi}^{B_s} + 4 U_{pi}^{A_t} U_{pi}^{B_s} \varepsilon_p + 4 U_{pi}^{A_t} U_{qj}^{B_s} A_{pi, qj}
\tag{1} \label{eq.1}
\end{align}

上面的记号与 Yamaguchi (V.2) 的不同之处除了使用了 Einstein Summation Convention，还有 $A, B$ 代表原子，而 $t, s$ 代表对应原子的坐标分量．我们可以很容易地知道，上述二阶梯度的维度是分别由两个原子与两个 3 维坐标分量构成．

### U 矩阵无关部分

从上述二阶梯度总公式 [(1)](#mjx-eqn-eq.1) 中，我们很容易看出第一行的前四项与分子轨道转换矩阵，即 U 矩阵，是无关的．我们将定义下面的记号

\begin{equation}
E_\mathrm{noU}^{A_t B_s} = 2 h_{ii}^{A_t B_s} + 2 (ii | jj)^{A_t B_s} - (ij | ij)^{A_t B_s} - 2 S_{ii}^{A_t B_s} \varepsilon_i
\tag{2}
\end{equation}

### U 矩阵相关部分

除开 U 矩阵无关部分的项的加和即是 U 矩阵相关部分：

\begin{equation}
E_\mathrm{withU}^{A_t B_s} = - 2 \eta_{ii}^{A_t B_s} \varepsilon_i + 4 U_{pi}^{B_s} F_{pi}^{A_t} + 4 U_{pi}^{A_t} F_{pi}^{B_s} + 4 U_{pi}^{A_t} U_{pi}^{B_s} \varepsilon_p + 4 U_{pi}^{A_t} U_{qj}^{B_s} A_{pi, qj}
\end{equation}

但是上式并非是真正用于计算的公式，其中的许多项可以被归并与重新计算．

第一，我们写出上式中 $\eta_{ii}^{A_t B_s}$ 的表达式 (Yamaguchi L.5)：

\begin{equation}
\eta_{ii}^{A_t B_s} = 2 U_{ip}^{A_t} U_{ip}^{B_s} - 2 S_{ip}^{A_t} S_{ip}^{B_s}
\tag{3} \label{eq.3}
\end{equation}

第二，占据-占据部分的 U 矩阵的值不被认为是确定的，但通常的做法是利用 U 矩阵的性质 (Yamaguchi J.2)，即 $S_{pq}^{A_t} + U_{pq}^{A_t} + U_{pq}^{A_t} = 0$，定义下述占据-占据部分的 U 矩阵

\begin{equation}
U_{ij}^{A_t} = U_{ji}^{A_t} = - \frac{1}{2} S_{ij}^{A_t}
\tag{4} \label{4}
\end{equation}

在 [一阶梯度](hf_nuc_grad.ipynb) 的文档中，我们指出 $\langle \nabla \mu | \nu \rangle$ 关于 $\mu \nu$ 是反对称矩阵；它也恰恰是 $S_{\mu \nu}^{A_t}$ 的基石；但 $S_{\mu \nu}^{A_t}$ 是对称矩阵，因为 $S_{\mu \nu}^{A_t} = - \langle \nabla \mu_A | \nu \rangle - \langle \mu | \nabla \nu_A \rangle$；若使用程序写出来，则对于第一个氧原子的 $x$ 分量的重叠积分导数矩阵则会写为

In [ ]:
tmp_s1ao = np.zeros((nao, nao))
tmp_s1ao[mol_slice(0)] -= int1e_ipovlp[0][mol_slice(0)]
tmp_s1ao[:, mol_slice(0)] -= int1e_ipovlp[0].T[:, mol_slice(0)]
np.allclose(tmp_s1ao, tmp_s1ao.T)

自然，如果 $S_{\mu \nu}^{A_t}$ 是对称矩阵，那么 $S_{pq}^{A_t}$ 也是对称的矩阵．

同时，根据 $S_{pq}^{A_t} + U_{pq}^{A_t} + U_{pq}^{A_t} = 0$ 的性质，并利用 $S_{pq}^{A_t}$ 的对称性，我们会将 [(3)](#mjx-eqn-eq.3) 中所有占据-非占的记号替换为非占-占据的记号，以与之后的记号匹配：

\begin{equation}
\eta_{ii}^{A_t B_s} = 2 U_{pi}^{A_t} S_{pi}^{B_s} + 2 U_{pi}^{B_s} S_{pi}^{A_t} + 2 U_{pi}^{A_t} U_{pi}^{B_s}
\tag{5} \label{5}
\end{equation}

第三，非占-占据部分的 U 矩阵的值通过 CP-HF 方程确定 (Yamaguchi X.1-3)：

\begin{equation}
U_{ai}^{A_t} (\varepsilon_i - \varepsilon_a) - A_{ai, bj} U_{bj}^{A_t} = F_{ai}^{A_t} - S_{ai}^{A_t} \varepsilon_i - \frac{1}{2} A_{ai, lj} S_{lj}^{A_t}
\tag{6} \label{6}
\end{equation}

上述的 CP-HF 方程与 [电子梯度的 CP-HF 方程](hf_elec_grad.ipynb#HF-极化率的导出) 的来源完全相同，但由于核梯度本身的复杂性，特别是 $S_{pq}^{A_t}$ 的存在，其形式与电子梯度相差稍大．

由于 [(4)](#mjx-eqn-eq.4) 的定义，我们可以将 $- \frac{1}{2} S_{lj}^{A_t}$ 用 $U_{lj}^{A_t}$ 替换；因此，[(6)](#mjx-eqn-eq.6) 还可以写作

\begin{equation}
U_{ai}^{A_t} (\varepsilon_a - \varepsilon_i) + A_{ai, qj} U_{qj}^{A_t} + F_{ai}^{A_t} - S_{ai}^{A_t} \varepsilon_i = 0
\tag{7} \label{eq.7}
\end{equation}

因此，根据上面的公式，我们对 $E_\mathrm{withU}^{A_t}$ 的形式书写如下：

\begin{align}
E_\mathrm{withU}^{A_t}
&= - 4 U_{pi}^{A_t} S_{pi}^{B_s} \varepsilon_i - 4 U_{pi}^{B_s} S_{pi}^{A_t} \varepsilon_i - 4 U_{pi}^{A_t} U_{pi}^{B_s} \varepsilon_i + 4 U_{pi}^{B_s} F_{pi}^{A_t} + 4 U_{pi}^{A_t} F_{pi}^{B_s} + 4 U_{pi}^{A_t} U_{pi}^{B_s} \varepsilon_p + 4 U_{pi}^{A_t} U_{qj}^{B_s} A_{pi, qj} \\
&= 4 U_{pi}^{A_t} (U_{pi}^{B_s} (\varepsilon_p - \varepsilon_i) + F_{pi}^{B_s} - S_{pi}^{B_s} \varepsilon_i + A_{pi, qj} U_{qj}^{B_s}) + 4 U_{pi}^{B_s} F_{pi}^{A_t} - 4 U_{pi}^{B_s} S_{pi}^{A_t} \varepsilon_i \\
&= -2 S_{ki}^{A_t} (U_{ki}^{B_s} (\varepsilon_k - \varepsilon_i) + F_{ki}^{B_s} - S_{ki}^{B_s} \varepsilon_i + A_{ki, qj} U_{qj}^{B_s}) + 4 U_{pi}^{B_s} F_{pi}^{A_t} - 4 U_{pi}^{B_s} S_{pi}^{A_t} \varepsilon_i
\tag{8} \label{eq.8}
\end{align}

等式一是代入 [(5)](#mjx-eqn-eq.5)．等式二则是整理公式．等式三首先利用 [(7)](#mjx-eqn-eq.7)，将第一项中按角标 $p$ 的占据与非占拆分；非占-占据部分 $U_{ai}^{A_t}$ 的项消为零，只留下占据-占据的部分 $U_{ki}^{A_t}$；随后对 $U_{ki}^{A_t}$ 使用 [(4)](#mjx-eqn-eq.4) 替换为 $- \frac{1}{2} S_{ki}^{A_t}$ 即得结果．

我们定义新的记号

\begin{equation}
M_{ki}^{B_s} = U_{ki}^{B_s} (\varepsilon_k - \varepsilon_i) + F_{ki}^{B_s} - S_{ki}^{B_s} \varepsilon_i + A_{ki, qj} U_{qj}^{B_s}
\tag{9} \label{eq.9}
\end{equation}

因此，

\begin{equation}
E_\mathrm{withU}^{A_t} = 4 U_{pi}^{B_s} F_{pi}^{A_t} - 4 U_{pi}^{B_s} S_{pi}^{A_t} \varepsilon_i - 2 S_{ki}^{A_t} M_{ki}^{B_s}
\tag{10} \label{eq.10}
\end{equation}

需要注意的是，$M_{ki}^{B_s}$ 的值是非零的；但如果将 占据轨道 $k$ 替换为非占轨道 $a$，那么 $M_{ai}^{B_s}$ 所代表的恰恰是 [(7)](#mjx-eqn-eq.7) 的等式右侧，为零值．之所以 $M_{ki}^{B_s}$ 通常是非零的，是因为 [(7)](#mjx-eqn-eq.7) 所代表的 CP-HF 方程只在非占-占据的 U 矩阵有意义．

至此，所有与矩阵有关的内容叙述完毕，剩下的是如何通过电子积分的计算生成这些矩阵，以及如何通过 CP-HF 生成 U 矩阵．

## U 矩阵无关部分

### 核哈密顿二阶导数

首先，我们考察 [(2)](#mjx-eqn-eq.2) 的第一项 $h_{ii}^{A_t B_s}$．与一阶导数相同地，这一项分为动能贡献与势能贡献．由于 $h_{ii}^{A_t B_s} = D_{\mu \nu} h_{\mu \nu}^{A_t B_s}$，我们直接考虑该核哈密顿在原子轨道下的矩阵表示：

\begin{align}
h_{\mu \nu}^{A_t B_s} 
&=
\partial_{A_t} \partial_{B_s} \langle \mu | \hat t | \nu \rangle +
\partial_{A_t} \partial_{B_s} \langle \mu | \hat v_\mathrm{nuc} | \nu \rangle
\tag{11} \label{eq.11}
\end{align}

我们会将上述两式拆分成四部分考虑．

第一部分是动能部分：

\begin{align}
t_{\mu \nu}^{A_t B_s} &= \partial_{A_t} \partial_{B_s} \langle \mu | \hat t | \nu \rangle \\ &=
\langle \partial_{A_t} \partial_{B_s} \mu | \hat t | \nu \rangle +
\langle \mu | \hat t | \partial_{A_t} \partial_{B_s} \nu \rangle +
\langle \partial_{A_t} \mu | \hat t | \partial_{B_s} \nu \rangle +
\langle \partial_{B_s} \mu | \hat t | \partial_{A_t} \nu \rangle \\ &=
\langle \partial_{r_t} \partial_{r_s} \mu_{AB} | \hat t | \nu \rangle +
\langle \mu | \hat t | \partial_{r_t} \partial_{r_s} \nu_{AB} \rangle +
\langle \partial_{r_t} \mu_A | \hat t | \partial_{r_s} \nu_B \rangle +
\langle \partial_{r_s} \mu_B | \hat t | \partial_{r_t} \nu_A \rangle
\tag{12} \label{eq.12}
\end{align}

上式中引入了双原子角标的记号 $\mu_{AB}$，它意味着，如果 $A$ 与 $B$ 等价，同时 $\mu$ 又是 $A$ 原子的原子轨道，那么这一项才有意义．注意到上面的推导中，$\partial_{A_t} \partial_{B_s} \mu = \partial_{r_t} \partial_{r_s} \mu_{AB}$；之所以符号是正号，是因为两次偏导数关系的负号负负得正．这对于 $\partial_{A_t} \mu \partial_{B_s} \nu = \partial_{r_t} \mu_A \partial_{r_s} \nu_B$ 也是相同的．

如果写成程序，一个比较麻烦的问题是矩阵的维度很难处理；它将是一个原子平方、三维空间平方、与原子轨道数平方的大小；或者对于当前问题，这个维度是

In [ ]:
(natm, natm, 3, 3, nao, nao)

尽管仍然无法减少计算量，但一种普遍在 PySCF 中的做法是构造一个函数，它在参数中传入原子序号，而输出 $(3, 3, n_\mathrm{AO}, n_\mathrm{AO})$ 大小的张量．以动能二阶导数为例，定义下述函数：

In [ ]:
def get_hess_ao_noU_kin(A, B):
    ao_matrix = np.zeros((3 * 3, nao, nao))
    zA, zB = mol.atom_charge(A), mol.atom_charge(B)
    sA, sB = mol_slice(A), mol_slice(B)
    if (A == B):
        ao_matrix[:, sA] += int1e_ipipkin[:, sA]
        ao_matrix[:, :, sA] += int1e_ipipkin[:, :, sA]
    ao_matrix[:, sA, sB] += int1e_ipipkin[:, sA, sB]
    ao_matrix[:, sB, sA] += int1e_ipipkin[:, sB, sA]
    return ao_matrix
# to return the tensor of kin hess of O1 and H2, run the following code:
# get_hess_ao_noU_kin(0, 3)

当然，[(12)](#mjx-eqn-eq.12) 中，项 $\langle \partial_{r_t} \partial_{r_s} \mu_{AB} | \hat t | \nu \rangle + \langle \partial_{r_t} \mu_A | \hat t | \partial_{r_s} \nu_B \rangle$ 与 $\langle \mu | \hat t | \partial_{r_t} \partial_{r_s} \nu_{AB} \rangle + \langle \partial_{r_s} \mu_B | \hat t | \partial_{r_t} \nu_A \rangle$ 互为矩阵 $\mu, \nu$ 的转置，因此上面的函数同样可以写为

In [ ]:
def get_hess_ao_noU_kin(A, B):
    ao_matrix = np.zeros((3 * 3, nao, nao))
    zA, zB = mol.atom_charge(A), mol.atom_charge(B)
    sA, sB = mol_slice(A), mol_slice(B)
    if (A == B):
        ao_matrix[:, sA] += int1e_ipipkin[:, sA]
    ao_matrix[:, sA, sB] += int1e_ipkinip[:, sA, sB]
    ao_matrix += ao_matrix.swapaxes(1, 2)
    return ao_matrix

代码上，需要补充说明的是，上述的输出维度是 $(9, n_\mathrm{AO}, n_\mathrm{AO})$；第一个维度代表两个三维维度的乘积．之后就不再说明．

第二部分是对轨道求导的势能二阶导数：

\begin{align}
{}^1 {} v_{\mu \nu}^{A_t B_s} &= 
\langle \partial_{A_t} \partial_{B_s} \mu | \hat v_\mathrm{nuc} | \nu \rangle +
\langle \mu | \hat v_\mathrm{nuc} | \partial_{A_t} \partial_{B_s} \nu \rangle +
\langle \partial_{A_t} \mu | \hat v_\mathrm{nuc} | \partial_{B_s} \nu \rangle +
\langle \partial_{B_s} \mu | \hat v_\mathrm{nuc} | \partial_{A_t} \nu \rangle \\ &=
\langle \partial_{r_t} \partial_{r_s} \mu_{AB} | \hat v_\mathrm{nuc} | \nu \rangle +
\langle \mu | \hat v_\mathrm{nuc} | \partial_{r_t} \partial_{r_s} \nu_{AB} \rangle +
\langle \partial_{r_t} \mu_A | \hat v_\mathrm{nuc} | \partial_{r_s} \nu_B \rangle +
\langle \partial_{r_s} \mu_B | \hat v_\mathrm{nuc} | \partial_{r_t} \nu_A \rangle
\tag{13} \label{eq.13}
\end{align}

上面的公式与动能非常类似，代码的写法也相同：

In [ ]:
def get_hess_ao_noU_vnuc1(A, B):
    ao_matrix = np.zeros((3 * 3, nao, nao))
    zA, zB = mol.atom_charge(A), mol.atom_charge(B)
    sA, sB = mol_slice(A), mol_slice(B)
    if (A == B):
        ao_matrix[:, sA] += int1e_ipipnuc[:, sA]
    ao_matrix[:, sA, sB] += int1e_ipnucip[:, sA, sB]
    ao_matrix += ao_matrix.swapaxes(1, 2)
    return ao_matrix

第三部分是对算符求导的势能二阶导数：

\begin{align}
{}^2 v_{\mu \nu}^{A_t B_s} &= \langle \mu | \partial_{A_t} \partial_{B_s} \hat v_\mathrm{nuc} | \nu \rangle
= \langle \mu | \partial_{r_t} \partial_{r_s} \frac{-Z_A}{r} | \nu \rangle_{r \rightarrow AB}
= \langle \frac{-Z_A}{r} | \partial_{r_t} \partial_{r_s} (\mu \nu) \rangle_{r \rightarrow AB} \\
&= \langle \partial_{r_t} \partial_{r_s} \mu | \frac{-Z_A}{r} | \nu \rangle_{r \rightarrow AB} +
\langle \mu | \frac{-Z_A}{r} | \partial_{r_t} \partial_{r_s} \nu \rangle_{r \rightarrow AB} +
\langle \partial_{r_t} \mu | \frac{-Z_A}{r} | \partial_{r_s} \nu \rangle_{r \rightarrow AB} +
\langle \partial_{r_s} \mu | \frac{-Z_A}{r} | \partial_{r_t} \nu \rangle_{r \rightarrow AB}
\tag{14} \label{eq.14}
\end{align}

上式中出现的 $r \rightarrow AB$ 代表的意义是，上式只在 $A$ 与 $B$ 原子等价时才成立，同时需要将坐标原点转换为 $A$ 原子的坐标．因此也意味着，若 $A$ 与 $B$ 原子不同，上式的贡献为零．其代码如下：

In [ ]:
def get_hess_ao_noU_vnuc2(A, B):
    ao_matrix = np.zeros((3 * 3, nao, nao))
    zA, zB = mol.atom_charge(A), mol.atom_charge(B)
    sA, sB = mol_slice(A), mol_slice(B)
    if (A == B):
        with mol.with_rinv_as_nucleus(A):
            ao_matrix -= zA * mol.intor('int1e_ipiprinv')
            ao_matrix -= zA * mol.intor('int1e_iprinvip')
    ao_matrix += ao_matrix.swapaxes(1, 2)
    return ao_matrix

第四部分是对算符求一次导，对轨道求一次导的二阶导数．这一步出现的项比较多，我们暂时取其中的一项进行分析：

\begin{align}
{}^3 v_{\mu \nu}^{A_t B_s} &\leftarrow 
\langle \partial_{A_t} \mu | \partial_{B_s} \hat v_\mathrm{nuc} | \nu \rangle
= \langle \partial_{r_t} \mu_A | \partial_{r_s} \frac{-Z_B}{r} | \nu \rangle_{r \rightarrow B}
= - \langle \frac{-Z_B}{r} | \partial_{r_s} (\partial_{r_t} \mu_A \nu) \rangle_{r \rightarrow B} \\
&= \langle \partial_{r_t} \partial_{r_s} \mu_A | \frac{Z_B}{r} | \nu \rangle_{r \rightarrow B} +
\langle \partial_{r_t} \mu_A | \frac{Z_B}{r} | \partial_{r_s} \nu \rangle_{r \rightarrow B}
\end{align}

注意到上式中，对算符的偏导数转为对轨道的偏导数过程中不像第三部分；这里取了负号，这是因为只有一个偏导数．上面对 ${}^3 v_{\mu \nu}^{A_t B_s}$ 总体贡献了两项，剩余的六项可以通过 $A_t, B_s$ 的互换、以及 $\mu, \nu$ 的转置即可得到．其代码书写如下：

In [ ]:
def get_hess_ao_noU_vnuc3(A, B):
    ao_matrix = np.zeros((3 * 3, nao, nao))
    zA, zB = mol.atom_charge(A), mol.atom_charge(B)
    sA, sB = mol_slice(A), mol_slice(B)
    with mol.with_rinv_as_nucleus(B):
        ao_matrix[:, sA] += zB * mol.intor('int1e_ipiprinv')[:, sA]
        ao_matrix[:, sA] += zB * mol.intor('int1e_iprinvip')[:, sA]
    with mol.with_rinv_as_nucleus(A):
        ao_matrix[:, sB] += zA * mol.intor('int1e_ipiprinv')[:, sB]
        ao_matrix[:, sB] += zA * mol.intor('int1e_iprinvip').swapaxes(1, 2)[:, sB]
        ao_matrix += ao_matrix.swapaxes(1, 2)
    return ao_matrix

到这里为止，我们已经完全构造好了核哈密顿量对二阶梯度的 AO 矩阵了．事实上，这部分在 PySCF 中的成员函数 `hessian.RHF.hcore_generator` 也可以生成；我们可以用下述的代码与 PySCF 的函数进行比对：

In [ ]:
[[ np.allclose(
    get_hess_ao_noU_kin(A, B) + get_hess_ao_noU_vnuc1(A, B) + get_hess_ao_noU_vnuc2(A, B) + get_hess_ao_noU_vnuc3(A, B),
    scf_hess.hcore_generator()(A, B).reshape(9, nao, nao)
) for B in range(natm) ] for A in range(natm)]

最后，我们计算 (角标 $i$ 被求和，因此下述张量是 $(A, B, t, s)$ 为角标的张量) 

\begin{equation}
2 h_{ii}^{A_t B_s} = h_{\mu \nu}^{A_t B_s} D_{\mu \nu}
\end{equation}

In [ ]:
def get_hess_noU_hcore(A, B):
    return np.einsum("Tuv, uv -> T",
                     get_hess_ao_noU_kin(A, B) + get_hess_ao_noU_vnuc1(A, B)
                     + get_hess_ao_noU_vnuc2(A, B) + get_hess_ao_noU_vnuc3(A, B), D).reshape(3, 3)

In [ ]:
hess_noU_hcore = np.array([ [ get_hess_noU_hcore(A, B) for B in range(natm) ] for A in range(natm) ])

注意上面代码中，循环的角标是先 $B$ 后 $A$．如果考察索引顺序的话，这个角标顺序就不难理解了．

### 双电子积分的导数

下面我们处理 [(2)](#mjx-eqn-eq.2) 中的项

\begin{equation}
2 (ii | jj)^{A_t B_s} - (ij | ij)^{A_t B_s} = \frac{1}{4} D_{\mu \nu} D_{\kappa \lambda} \big( 2 (\mu \nu | \kappa \lambda)^{A_t B_s} - (\mu \kappa | \nu \lambda)^{A_t B_s} \big) = \frac{1}{4} (2 D_{\mu \nu} D_{\kappa \lambda} - D_{\mu \kappa} D_{\nu \lambda}) (\mu \nu | \kappa \lambda)^{A_t B_s}
\end{equation}

这一项会产生许多小项，公式尽管不难但多且繁杂．我们一点一点来处理．

首先是库伦积分：

\begin{align}
D_{\mu \nu} D_{\kappa \lambda} (\mu \nu | \kappa \lambda)^{A_t B_s} 
&= 
D_{\mu \nu} D_{\kappa \lambda} \big(
(\partial_{r_t} \partial_{r_s} \mu_{AB} \nu | \kappa \lambda) +
(\mu \partial_{r_t} \partial_{r_s} \nu_{AB} | \kappa \lambda) +
(\mu \nu | \partial_{r_t} \partial_{r_s} \kappa_{AB} \lambda) +
(\mu \nu | \kappa \partial_{r_t} \partial_{r_s} \lambda_{AB})
\big) \\ &\quad +
D_{\mu \nu} D_{\kappa \lambda} \big(
(\partial_{r_t} \mu_{A} \partial_{r_s} \nu_{B} | \kappa \lambda) +
(\partial_{r_s} \mu_{B} \partial_{r_t} \nu_{A} | \kappa \lambda) +
(\mu \nu | \partial_{r_t} \kappa_{A} \partial_{r_s} \lambda_{B}) +
(\mu \nu | \partial_{r_s} \kappa_{B} \partial_{r_t} \lambda_{A})
\big) \\ &\quad +
D_{\mu \nu} D_{\kappa \lambda} \big(
(\partial_{r_t} \mu_A \nu | \partial_{r_s} \kappa_B \lambda) +
(\partial_{r_t} \mu_A \nu | \kappa \partial_{r_s} \lambda_B) +
(\mu \partial_{r_t} \nu_A | \partial_{r_s} \kappa_B \lambda) +
(\mu \partial_{r_t} \nu_A | \kappa \partial_{r_s} \lambda_B)
\big) \\ &\quad +
D_{\mu \nu} D_{\kappa \lambda} \big(
(\partial_{r_s} \mu_B \nu | \partial_{r_t} \kappa_A \lambda) +
(\partial_{r_s} \mu_B \nu | \kappa \partial_{r_t} \lambda_B) +
(\mu \partial_{r_s} \nu_B | \partial_{r_t} \kappa_A \lambda) +
(\mu \partial_{r_s} \nu_B | \kappa \partial_{r_t} \lambda_B)
\big)
\\ &= 
2 D_{\mu \nu} D_{\kappa \lambda} \big(
(\partial_{r_t} \partial_{r_s} \mu_{AB} \nu | \kappa \lambda) +
(\mu \partial_{r_t} \partial_{r_s} \nu_{AB} | \kappa \lambda)
\big) \\ &\quad +
2 D_{\mu \nu} D_{\kappa \lambda} \big(
(\partial_{r_t} \mu_{A} \partial_{r_s} \nu_{B} | \kappa \lambda) +
(\partial_{r_s} \mu_{B} \partial_{r_t} \nu_{A} | \kappa \lambda)
\big) \\ &\quad +
2 D_{\mu \nu} D_{\kappa \lambda} \big(
(\partial_{r_t} \mu_A \nu | \partial_{r_s} \kappa_B \lambda) +
(\partial_{r_t} \mu_A \nu | \kappa \partial_{r_s} \lambda_B) +
(\mu \partial_{r_t} \nu_A | \partial_{r_s} \kappa_B \lambda) +
(\mu \partial_{r_t} \nu_A | \kappa \partial_{r_s} \lambda_B)
\big)
\tag{15} \label{eq.15}
\end{align}

上式的第二个等号利用 $\kappa, \lambda$ 与 $\mu, \nu$ 的角标互换．

我们先分析 [(15)](#mjx-eqn-eq.15) 第二个等号后的第一行．该行对应 PySCF 积分引擎的 `int2e_ipip1` $(\partial_{r_t} \partial_{r_s} \mu \nu | \kappa \lambda)$．由于 PySCF 积分引擎中，轨道的顺序是确定的．以写到 PySCF 积分引擎的积分顺序为前提，简化上式如下：

\begin{align}
&\quad 2 D_{\mu \nu} D_{\kappa \lambda} (\partial_{r_t} \partial_{r_s} \mu_{AB} \nu | \kappa \lambda) +
2 D_{\mu \nu} D_{\kappa \lambda}(\mu \partial_{r_t} \partial_{r_s} \nu_{AB} | \kappa \lambda) \\
&= 2 \big( D_{\mu_{AB} \nu} + D_{\nu \mu_{AB}} \big) D_{\kappa \lambda} (\partial_{r_t} \partial_{r_s} \mu_{AB} \nu | \kappa \lambda) \\
&= 4 D_{\mu_{AB} \nu} D_{\kappa \lambda} (\partial_{r_t} \partial_{r_s} \mu_{AB} \nu | \kappa \lambda)
\tag{15.1} \label{eq.15.1}
\end{align}

其中第一个等号是 $\mu, \nu$ 角标互换，第二个等号利用密度矩阵的对称性．

随后是 [(15)](#mjx-eqn-eq.15) 第二个等号后的第二行；它对应 PySCF 积分引擎的 `int2e_ipvip1` $(\partial_{r_t} \mu \partial_{r_s} \nu | \kappa \lambda)$；其简化过程与上面相同：

\begin{align}
&\quad 2 D_{\mu \nu} D_{\kappa \lambda} (\partial_{r_t} \mu_{A} \partial_{r_s} \nu_{B} | \kappa \lambda) +
2 D_{\mu \nu} D_{\kappa \lambda} (\partial_{r_s} \mu_{B} \partial_{r_t} \nu_{A} | \kappa \lambda) \\
&= 2 \big( D_{\mu_A \nu_B} + D_{\nu_B \mu_A} \big) D_{\kappa \lambda} (\partial_{r_t} \mu_{A} \partial_{r_s} \nu_{B} | \kappa \lambda) \\
&= 4 D_{\mu_A \nu_B} D_{\kappa \lambda} (\partial_{r_t} \mu_{A} \partial_{r_s} \nu_{B} | \kappa \lambda)
\tag{15.2} \label{eq.15.2}
\end{align}

[(15)](#mjx-eqn-eq.15) 第二个等号后的第三行对应 PySCF 积分引擎的 `int2e_ip1ip2` $(\partial_{r_t} \mu \nu | \partial_{r_s} \kappa \lambda)$；简化过程仍然是相同的：

\begin{align}
&\quad
2 D_{\mu \nu} D_{\kappa \lambda} \big(
(\partial_{r_t} \mu_A \nu | \partial_{r_s} \kappa_B \lambda) +
(\partial_{r_t} \mu_A \nu | \kappa \partial_{r_s} \lambda_B) +
(\mu \partial_{r_t} \nu_A | \partial_{r_s} \kappa_B \lambda) +
(\mu \partial_{r_t} \nu_A | \kappa \partial_{r_s} \lambda_B)
\big) \\ &=
2 \big( D_{\mu_A \nu} D_{\kappa_B \lambda} + D_{\mu_A \nu} D_{\lambda \kappa_B} + D_{\nu \mu_A} D_{\kappa_B \lambda} + D_{\nu \mu_A} D_{\lambda \kappa_B} \big) (\partial_{r_t} \mu_A \nu | \partial_{r_s} \kappa_B \lambda) \\
&= 8 D_{\mu_A \nu} D_{\kappa_B \lambda} (\partial_{r_t} \mu_A \nu | \partial_{r_s} \kappa_B \lambda)
\tag{15.3} \label{eq.15.3}
\end{align}

随后我们考虑交换积分．我们考虑 $\mu, \kappa$ 与 $\nu \lambda$ 角标的相互对调后，应当能得到

\begin{align}
D_{\mu \kappa} D_{\nu \lambda} (\mu \nu | \kappa \lambda)^{A_t B_s} 
&=
2 D_{\mu \kappa} D_{\nu \lambda} \big(
(\partial_{r_t} \partial_{r_s} \mu_{AB} \nu | \kappa \lambda) +
(\mu \nu | \partial_{r_t} \partial_{r_s} \kappa_{AB} \lambda)
\big) \\ &\quad +
2 D_{\mu \kappa} D_{\nu \lambda} \big(
(\partial_{r_t} \mu_{A} \partial_{r_s} \nu_{B} | \kappa \lambda) +
(\mu \nu | \partial_{r_t} \kappa_{A} \partial_{r_s} \lambda_{B})
\big) \\ &\quad +
2 D_{\mu \kappa} D_{\nu \lambda} \big(
(\partial_{r_t} \mu_A \nu | \partial_{r_s} \kappa_B \lambda) +
(\partial_{r_t} \mu_A \nu | \kappa \partial_{r_s} \lambda_B) +
(\partial_{r_s} \mu_B \nu | \partial_{r_t} \kappa_A \lambda) +
(\partial_{r_s} \mu_B \nu | \kappa \partial_{r_t} \lambda_A)
\big)
\tag{16} \label{eq.16}
\end{align}

仿照对 [(15)](#mjx-eqn-eq.15) 简化的过程，我们应当可以对上式作简化；其结果为：

\begin{align}
D_{\mu \kappa} D_{\nu \lambda} (\mu \nu | \kappa \lambda)^{A_t B_s}
&= 4 D_{\mu_{AB} \kappa} D_{\nu \lambda} (\partial_{r_t} \partial_{r_s} \mu_{AB} \nu | \kappa \lambda) +
4 D_{\mu_A \kappa} D_{\nu_B \lambda} (\partial_{r_t} \mu_A \partial_{r_s} \nu_B | \kappa \lambda) \\ &\quad +
4 D_{\mu_A \kappa_B} D_{\nu \lambda} (\partial_{r_t} \mu_A \nu | \partial_{r_s} \kappa_B \lambda) +
4 D_{\mu_A \kappa} D_{\nu \lambda_B} (\partial_{r_t} \mu_A \nu | \kappa \partial_{r_s} \lambda_B)
\tag{16.1} \label{eq.16.1}
\end{align}

为了与 [(15.1)](#mjx-eqn-eq.15.1), [(15.2)](#mjx-eqn-eq.15.2), [(15.3)](#mjx-eqn-eq.15.3) 在密度矩阵上的记号相近，我们更改上面公式的角标 (对调角标) 如下：

\begin{align}
D_{\mu \kappa} D_{\nu \lambda} (\mu \nu | \kappa \lambda)^{A_t B_s}
&= 4 D_{\mu_{AB} \nu} D_{\kappa \lambda} (\partial_{r_t} \partial_{r_s} \mu_{AB} \kappa | \nu \lambda) +
4 D_{\mu_A \nu} D_{\kappa_B \lambda} (\partial_{r_t} \mu_A \partial_{r_s} \kappa_B | \nu \lambda) \\ &\quad +
4 D_{\mu_A \nu_B} D_{\kappa \lambda} (\partial_{r_t} \mu_A \kappa | \partial_{r_s} \nu_B \lambda) +
4 D_{\mu_A \nu} D_{\kappa_B \lambda} (\partial_{r_t} \mu_A \lambda | \partial_{r_s} \kappa_B \nu)
\tag{16.2} \label{eq.16.2}
\end{align}

最后，我们合并 [(15.1)](#mjx-eqn-eq.15.1), [(15.2)](#mjx-eqn-eq.15.2), [(15.3)](#mjx-eqn-eq.15.3), [(16.1)](#mjx-eqn-eq.16.1) 入双电子积分导数，并作简单的合并同类项，得到最终的结果：

\begin{align}
2 (ii | jj)^{A_t B_s} - (ij | ij)^{A_t B_s} &=
D_{\mu_{AB} \nu} \big( 2 (\partial_{r_t} \partial_{r_s} \mu_{AB} \nu | \kappa \lambda) - (\partial_{r_t} \partial_{r_s} \mu_{AB} \kappa | \nu \lambda) \big) D_{\kappa \lambda} \\ &\quad +
D_{\mu_A \nu_B} \big( 2 (\partial_{r_t} \mu_{A} \partial_{r_s} \nu_{B} | \kappa \lambda) - (\partial_{r_t} \mu_A \kappa | \partial_{r_s} \nu_B \lambda) \big) D_{\kappa \lambda} \\ &\quad +
D_{\mu_A \nu} D_{\kappa_B \lambda} \big( 4 (\partial_{r_t} \mu_A \nu | \partial_{r_s} \kappa_B \lambda) - (\partial_{r_t} \mu_A \partial_{r_s} \kappa_B | \nu \lambda) - (\partial_{r_t} \mu_A \lambda | \partial_{r_s} \kappa_B \nu) \big) 
\tag{17} \label{eq.17}
\end{align}

对上式，其代码可以书写如下；

In [ ]:
def get_hess_noU_eri_slow(A, B):
    hess_matrix = np.zeros((3 * 3))
    sA, sB = mol_slice(A), mol_slice(B)
    if (A == B):
        hess_matrix += np.einsum("Tuvkl, uv, kl -> T", int2e_ipip1[:, sA],         D[sA],     D    ) * 2
        hess_matrix -= np.einsum("Tukvl, uv, kl -> T", int2e_ipip1[:, sA],         D[sA],     D    )
    hess_matrix +=     np.einsum("Tuvkl, uv, kl -> T", int2e_ipvip1[:, sA, sB],    D[sA, sB], D    ) * 2
    hess_matrix -=     np.einsum("Tukvl, uv, kl -> T", int2e_ip1ip2[:, sA, :, sB], D[sA, sB], D    )
    hess_matrix +=     np.einsum("Tuvkl, uv, kl -> T", int2e_ip1ip2[:, sA, :, sB], D[sA],     D[sB]) * 4
    hess_matrix -=     np.einsum("Tukvl, uv, kl -> T", int2e_ipvip1[:, sA, sB],    D[sA],     D[sB])
    hess_matrix -=     np.einsum("Tulkv, uv, kl -> T", int2e_ip1ip2[:, sA, :, sB], D[sA],     D[sB])
    return hess_matrix.reshape(3, 3)

但上面的计算会较为耗时；我们可以预先储存一些矩阵与张量以加快速度，但以牺牲代码可读性为代价．其中，`eri_mat1`, `eri_mat2`, `eri_tensor1` 分别代表式 [(17)](#mjx-eqn-eq.17) 的第一到三行的矩阵或张量．

In [ ]:
eri_mat1 = np.einsum("Tuvkl, kl -> Tuv", int2e_ipip1, D) * 2 - np.einsum("Tukvl, kl -> Tuv", int2e_ipip1, D)
eri_mat2 = np.einsum("Tuvkl, kl -> Tuv", int2e_ipvip1, D) * 2 - np.einsum("Tukvl, kl -> Tuv", int2e_ip1ip2, D)
eri_tensor1 = int2e_ip1ip2 * 4 - int2e_ipvip1.swapaxes(2, 3) - int2e_ip1ip2.swapaxes(2, 4)

def get_hess_noU_eri_direct(A, B):
    hess_matrix = np.zeros((3 * 3))
    sA, sB = mol_slice(A), mol_slice(B)
    if (A == B):
        hess_matrix += np.einsum("Tuv, uv -> T",       eri_mat1[:, sA],           D[sA])
    hess_matrix +=     np.einsum("Tuv, uv -> T",       eri_mat2[:, sA, sB],       D[sA, sB])
    hess_matrix +=     np.einsum("Tuvkl, uv, kl -> T", eri_tensor1[:, sA, :, sB], D[sA], D[sB])
    return hess_matrix.reshape(3, 3)

另一种做法是像 [核哈密顿](#核哈密顿二阶导数) 的做法一样，先生成一个原子轨道的矩阵模样返回值的函数 (但不是真正的原子轨道下的矩阵，这仅仅是为了计算方便而用；具体的原因参考 [下文](#Fock-矩阵一阶导数) 的说明)，再将这个矩阵与密度相乘最终得到二阶梯度．代码也并不复杂：

In [ ]:
def get_hess_ao_noU_eri(A, B):
    ao_matrix = np.zeros((9, nao, nao))
    sA, sB = mol_slice(A), mol_slice(B)
    if (A == B):
        ao_matrix[:, sA] += eri_mat1[:, sA]
    ao_matrix[:, sA, sB] += eri_mat2[:, sA, sB]
    ao_matrix[:, sA] += np.einsum("Tuvkl, kl -> Tuv", eri_tensor1[:, sA, :, sB], D[sB])
    return ao_matrix

def get_hess_noU_eri(A, B):
    return np.einsum("Tuv, uv -> T", get_hess_ao_noU_eri(A, B), D).reshape(3, 3)

我们可以验证这三种函数给出的对二阶梯度的贡献结果是相同的：

In [ ]:
hess_noU_eri_slow =   np.array([ [ get_hess_noU_eri_slow(A, B)   for B in range(natm) ] for A in range(natm) ])
hess_noU_eri_direct = np.array([ [ get_hess_noU_eri_direct(A, B) for B in range(natm) ] for A in range(natm) ])
hess_noU_eri =        np.array([ [ get_hess_noU_eri(A, B)        for B in range(natm) ] for A in range(natm) ])
print(np.allclose(hess_noU_eri_slow, hess_noU_eri))
print(np.allclose(hess_noU_eri_direct, hess_noU_eri))

同时，我们可以简单地测试一下提速量：

In [ ]:
%%timeit -r 5 -n 5
[ [ get_hess_noU_eri_slow(A, B) for B in range(natm) ] for A in range(natm) ]

In [ ]:
%%timeit -r 5 -n 5
[ [ get_hess_noU_eri_direct(A, B) for B in range(natm) ] for A in range(natm) ]

In [ ]:
%%timeit -r 5 -n 5
[ [ get_hess_noU_eri(A, B) for B in range(natm) ] for A in range(natm) ]

### 重叠积分有关导数

最后我们处理 [(2)](#mjx-eqn-eq.2) 的 $- 2 S_{ii}^{A_t B_s} \varepsilon_i$．这一项的处理较为方便：

\begin{align}
- 2 S_{ii}^{A_t B_s} \varepsilon_i = - 2 C_{\mu i} \varepsilon_i C_{\nu i} S_{\mu \nu}^{A_t B_s} &= -
D_{\mu \nu} [\boldsymbol{\varepsilon}] \big(
\langle \partial_{A_t} \partial_{B_s} \mu | \nu \rangle +
\langle \mu | \partial_{A_t} \partial_{B_s} \nu \rangle +
\langle \partial_{A_t} \mu | \partial_{B_s} \nu \rangle +
\langle \partial_{B_s} \mu | \partial_{A_t} \nu \rangle
\big) \\ &=
- 2 D_{\mu \nu} [\boldsymbol{\varepsilon}] \big( \langle \partial_{r_t} \partial_{r_s} \mu_{AB} | \nu \rangle +
[\boldsymbol{\varepsilon}] \langle \partial_{r_t} \mu_A | \partial_{r_s} \nu_B \rangle \big)
\end{align}

与上面代码的区别仅仅是需要使用能量加权密度替换普通的 SCF 密度．

In [ ]:
def get_hess_ao_noU_S(A, B):
    ao_matrix = np.zeros((9, nao, nao))
    sA, sB = mol_slice(A), mol_slice(B)
    if (A == B):
        ao_matrix[:, sA] -= int1e_ipipovlp[:, sA] * 2
    ao_matrix[:, sA, sB] -= int1e_ipovlpip[:, sA, sB] * 2
    return ao_matrix

def get_hess_noU_S(A, B):
    return np.einsum("Tuv, uv -> T", get_hess_ao_noU_S(A, B), De).reshape((3, 3))

In [ ]:
hess_noU_S = np.array([ [ get_hess_noU_S(A, B) for B in range(natm) ] for A in range(natm) ])

### U 矩阵无关部分：总结

至此，我们已经完成了所有与 U 矩阵无关的二阶梯度的部分．我们简单地将每个分项的贡献加和：

In [ ]:
hess_noU = hess_noU_hcore + hess_noU_eri + hess_noU_S

事实上，PySCF 使用了一个函数生成这些项：

In [ ]:
np.allclose(hess_noU, scf_hess.partial_hess_elec())

## U 矩阵相关部分

随后我们需要考虑 U 矩阵相关的部分，即计算式 [(10)](#mjx-eqn-eq.10)．为了计算该式，需要解 CP-HF 方程 [(7)](#mjx-eqn-eq.7)．为了给出 CP-HF 方程的所有张量，我们需要求 Fock 矩阵的一阶导数 $F_{ai}^{A_t}$、重叠积分的一阶导数 $S_{ai}^{A_t}$，并且定义张量乘积 $A_{ai, qj} U_{qj}^{A_t}$．

### 回顾：一阶梯度的导数矩阵

Fock 矩阵的一阶导数实际上几乎就是构建一阶梯度所产生的原子轨道角标的张量．我们先回顾一下一阶梯度的构建方式．在 [一阶梯度代码总结](hf_nuc_grad.ipynb/#代码总结) 中，我们直接构建了梯度；在这里，我们稍微绕个弯，先生成一个原子轨道角标的张量，再通过与密度矩阵相乘得到最终结果：(下述的函数使用全局变量计算梯度，而不是传入 SCF 类信息)

In [ ]:
def grad_ao_contrib1(A):
    ao_matrix = np.zeros((3, nao, nao))
    sA = mol_slice(A)
    ao_matrix[:, sA] = (- int1e_ipkin - int1e_ipnuc 
                        - 0.5 * np.einsum("tuvkl, kl -> tuv", int2e_ip1, D)
                        + 0.25 * np.einsum("tukvl, kl -> tuv", int2e_ip1, D)
                       )[:, sA]
    ao_matrix -= 0.5 * np.einsum("tkluv, kl -> tuv", int2e_ip1[:, sA], D[sA])
    ao_matrix += 0.25 * np.einsum("tkulv, kl -> tuv", int2e_ip1[:, sA], D[sA])
    with mol.with_rinv_as_nucleus(A):
        ao_matrix -= mol.intor("int1e_iprinv") * mol.atom_charge(A)
    return ao_matrix + ao_matrix.swapaxes(1, 2)

def grad_ao_contrib2(A):
    ao_matrix = np.zeros((3, nao, nao))
    sA = mol_slice(A)
    ao_matrix[:, sA] = -int1e_ipovlp[:, sA]
    return ao_matrix + ao_matrix.swapaxes(1, 2)

def my_grad_elec():
    grad_matrix = np.zeros((natm, 3))
    for A in range(natm):
        grad_matrix[A] += np.einsum("tuv, uv -> t", grad_ao_contrib1(A), D)
        grad_matrix[A] -= np.einsum("tuv, uv -> t", grad_ao_contrib2(A), De)
    return grad_matrix

np.allclose(my_grad_elec(), scf_grad.grad_elec())

在上面的代码中，`grad_ao_contrib1` 所作的是给出关于 $t, \mu, \nu$ 的张量 (下式对 $\kappa, \tau$ 求和)

\begin{align}
&\quad 2 h_{\mu \nu}^{A_t} + (\mu \nu | \kappa \tau)^{A_t} D_{\kappa \tau} - \frac{1}{2} (\mu \kappa | \nu \lambda)^{A_t} D_{\kappa \tau} \\
&= h_{\mu \nu}^{A_t} + \big( (\partial_{r_t} \mu_A \nu | \kappa \tau) + (\partial_{r_t} \kappa_A \lambda | \mu \nu) \big) D_{\kappa \tau} -
\frac{1}{2} \big( (\partial_{r_t} \mu_A \kappa | \nu \tau) + (\partial_{r_t} \kappa_A \mu | \lambda \nu) \big) D_{\kappa \tau} + \mathrm{interchange} (\mu, \nu)
\end{align}

在以前的代码中，由于我们直接计算了梯度张量，因此以上式中的库伦积分导数为例 (下式对 $\mu, \nu, \kappa, \tau$ 求和)

\begin{equation}
(\partial_{r_t} \mu_A \nu | \kappa \tau) D_{\mu \nu} D_{\kappa \tau} = (\partial_{r_t} \kappa_A \lambda | \mu \nu) \big) D_{\mu \nu} D_{\kappa \tau}
\end{equation}

但上面等号的成立是通过交换 $\kappa, \lambda$ 与 $\mu, \nu$ 成立的；如果上式只针对 $\kappa, \tau$ 求和，那么作为结果的关于 $t, \mu, \nu$ 的张量就不相等了．

由于求梯度本身会对所有原子轨道求和，因此我们对角标的顺序、矩阵的性质都不很关心．举例子来讲，我们观察下述代码：

In [ ]:
def grad_ao_contrib1_fake(A):
    ao_matrix = np.zeros((3, nao, nao))
    sA = mol_slice(A)
    ao_matrix[:, sA] = (- int1e_ipkin - int1e_ipnuc 
                        - 1 * np.einsum("tuvkl, kl -> tuv", int2e_ip1, D)
                        + 0.5 * np.einsum("tukvl, kl -> tuv", int2e_ip1, D)
                       )[:, sA]
    with mol.with_rinv_as_nucleus(A):
        ao_matrix -= mol.intor("int1e_iprinv") * mol.atom_charge(A)
    return ao_matrix * 2

尽管上述函数对 $D_{\mu \nu}$ 相乘并求和后所产生的对一阶梯度的贡献，与 `grad_ao_contrib1` 函数是一样的：

In [ ]:
np.allclose(
    np.array([ np.einsum("tuv, uv -> t", grad_ao_contrib1(A), D) for A in range(natm) ]),
    np.array([ np.einsum("tuv, uv -> t", grad_ao_contrib1_fake(A), D) for A in range(natm) ])
)

但这两个函数生成的矩阵实际上差别很大；拿第一个原子来看：

In [ ]:
np.allclose(grad_ao_contrib1(0), grad_ao_contrib1_fake(0))

### Fock 矩阵一阶导数

就像刚才生成梯度的关于 $t, \mu, \nu$ 张量一样，Fock 矩阵的一阶导数是关于 $t, \mu, \nu$ 的张量，因此也要按照生成 `grad_ao_contrib1` 的方式来生成 $F_{\mu \nu}^{A_t}$．事实上，Fock 矩阵的一阶导数相对于梯度的关于 $t, \mu, \nu$ 张量的区别仅仅是在双电子导数上多了一倍，因此构造 $F_{\mu \nu}^{A_t}$ 的方式与梯度的张量完全相同．

In [ ]:
def get_hess_ao_h1(A):
    ao_matrix = np.zeros((3, nao, nao))
    sA = mol_slice(A)
    ao_matrix[:, sA] = (- int1e_ipkin - int1e_ipnuc 
                        - np.einsum("tuvkl, kl -> tuv", int2e_ip1, D)
                        + 0.5 * np.einsum("tukvl, kl -> tuv", int2e_ip1, D)
                       )[:, sA]
    ao_matrix -= np.einsum("tkluv, kl -> tuv", int2e_ip1[:, sA], D[sA])
    ao_matrix += 0.5 * np.einsum("tkulv, kl -> tuv", int2e_ip1[:, sA], D[sA])
    with mol.with_rinv_as_nucleus(A):
        ao_matrix -= mol.intor("int1e_iprinv") * mol.atom_charge(A)
    return ao_matrix + ao_matrix.swapaxes(1, 2)

hess_ao_h1 = np.array([ get_hess_ao_h1(A) for A in range(natm) ])

Fock 矩阵一阶导数在 PySCF 中也有函数定义，我们可以验证上面函数是正确的：

In [ ]:
np.allclose(scf_hess.make_h1(C, mo_occ), hess_ao_h1)

自然，非占-占据分子轨道下的 Fock 矩阵一阶导数可由 $F_{ai}^{A_t} = C_{\mu a} F_{\mu \nu}^{A_t} C_{\nu i}$ 生成．不过需要指出的是，在使用 PySCF 解 CP-HF 方程时，我们将会使用的并非非占-占据的 $F_{ai}^{A_t}$，而是全轨道-占据的 $F_{pi}^{A_t}$．

In [ ]:
hess_pi_h1 = np.einsum("Atuv, up, vi -> Atpi", hess_ao_h1, C, Co)

### 重叠矩阵的一阶导数

我们甚至已经不需要再定义新的原子轨道下重叠矩阵的一阶导数 $S_{\mu \nu}^{A_t}$；这与一阶梯度的实现中定义的函数完全相同：

In [ ]:
get_hess_ao_s1 = grad_ao_contrib2
hess_ao_s1 = np.array([ get_hess_ao_s1(A) for A in range(natm) ])

同样地，在使用 PySCF 解 CP-HF 方程时，我们将会使用全轨道-占据的 $S_{pi}^{A_t}$．

In [ ]:
hess_pi_s1 = np.einsum("Atuv, up, vi -> Atpi", hess_ao_s1, C, Co)

### $A_{pi, qj} x_{qj}^a$ 的构造

这里的构造与 [电性质导数](hf_elec_grad.ipynb#避免四脚标张量-$A_{ai,-bj}$) 中 $A_{ai, bj} x_{bj}$ 的构造完全相同，唯一的区别是现在的角标是全轨道-占据而非非占-占据；同时，$x_{qj}^a$ 将不再是二角标矩阵，而是张量．

In [ ]:
def Ax(x):
    shape = x.shape
    x = x.reshape((-1, nmo, nocc))
    dx = C @ x @ Co.T
    v = np.zeros_like(x)
    for i in range(dx.shape[0]):
        v[i] = C.T @ scf_eng.get_veff(mol, dx[i] + dx[i].T) @ Co
    return 2 * v.reshape(shape)

在 PySCF 中有执行相同任务的函数；我们可以相互验证：

In [ ]:
fx = hessian.rhf.gen_vind(scf_eng, C, mo_occ)
np.allclose(Ax(hess_pi_h1.reshape(-1, nmo, nocc)), fx(hess_pi_h1))

### CP-HF 方程求解

在这里，我们不再像 [电性质导数](hf_elec_grad.ipynb#CP-方程的迭代求解) 那边一样手动求解 CP 方程，只是我们会简单地验证结果的性质．我们使用传入五参数的 `scf.cphf.solve` 来解 CP-HF 方程 (电性质导数由于不需要传入 $S_{ai}^{A_t}$，因此调用时最少传入四参数；这里指的参数是非可选参数)．

In [ ]:
hess_U, hess_M = scf.cphf.solve(Ax, e, mo_occ, hess_pi_h1.reshape(-1, nmo, nocc), hess_pi_s1.reshape(-1, nmo, nocc))
hess_U.shape = (natm, 3, nmo, nocc); hess_M.shape = (natm, 3, nocc, nocc)

传入五参数的 `scf.cphf.solve` 与传入四参数的 CP-HF 不同，五参数函数将会输出两项，$U_{pi}^{A_t}$ 与 $M_{ki}^{A_t}$．

对于上述的结果，在此我们用式 [(4)](#mjx-eqn-eq.4), [(7)](#mjx-eqn-eq.7), [(9)](#mjx-eqn-eq.9) 验证：

In [ ]:
# Equation (4) : U_ij = - 0.5 S_ij
np.allclose(hess_U[:, :, :nocc], - 0.5 * hess_pi_s1[:, :, :nocc])

In [ ]:
hess_M_tmp = hess_U * lib.direct_sum("a - i", e, eo) + Ax(hess_U) + hess_pi_h1 - hess_pi_s1 * eo

In [ ]:
# Equation (7) : CP-HF equation for virt-occ part
np.allclose(hess_M_tmp[:, :, nocc:], np.zeros((natm, 3, nvir, nocc)), atol=1e-7)

In [ ]:
# Equation (9) : self-defined M matrix for occ-occ part
np.allclose(hess_M_tmp[:, :, :nocc], hess_M)

### U 矩阵相关部分：总结

在求出 U 矩阵与自定义的 M 矩阵后，我们可以依靠式 [(10)](#mjx-eqn-eq.10) 生成 U 矩阵相关部分的二阶梯度：

In [ ]:
hess_withU = 4 * np.einsum("Bspi, Atpi -> ABts", hess_U, hess_pi_h1)
hess_withU -= 4 * np.einsum("Bspi, Atpi, i -> ABts", hess_U, hess_pi_s1, eo)
hess_withU -= 2 * np.einsum("Atki, Bski -> ABts", hess_pi_s1[:, :, :nocc], hess_M)

至此，我们已经完全求得了 [(1)](#mjx-eqn-eq.1) 二阶梯度的电子能量部分了．我们将其加和，并与 PySCF 中生成电子能量部分梯度的输出进行比较：

In [ ]:
np.allclose(hess_withU + hess_noU, scf_hess.hess_elec())

## 原子核斥力的二阶梯度

原子核斥力的二阶梯度可以被表示为

\begin{align}
\frac{\partial^2 E_\mathrm{nuc}}{\partial_{A_t} \partial_{B_s}} = 
- 3 \frac{Z_A Z_B}{| \boldsymbol{A} - \boldsymbol{B} |^5} (A_t - B_t) (A_s - B_s)
+ 3 \frac{Z_A Z_M}{| \boldsymbol{A} - \boldsymbol{M} |^5} (A_t - M_t) (A_s - M_s)
+ \frac{Z_A Z_B}{| \boldsymbol{A} - \boldsymbol{B} |^3}
- \frac{Z_A Z_M}{| \boldsymbol{A} - \boldsymbol{M} |^3}
\tag{18} \label{eq.18}
\end{align}

我们可以使用一些简化的记号．如果我们定义一些方便的记号：

\begin{align}
Z_{MN} &= Z_M Z_N \\
V_{MNt} &= M_t - N_t \\
r_{MN} &= | \boldsymbol{M} - \boldsymbol{N} |
\end{align}

那么就可以方便地使用张量计算的函数、以及一些矩阵索引的技巧来在较短的代码中实现上述过程．

In [ ]:
nuc_Z = np.einsum("M, N -> MN", mol.atom_charges(), mol.atom_charges())
nuc_V = lib.direct_sum("Mt - Nt -> MNt", mol.atom_coords(), mol.atom_coords())
nuc_rinv = 1 / (np.linalg.norm(nuc_V, axis=2) + np.diag([np.inf] * natm))

式 [(18)](#mjx-eqn-eq.18) 可以简记为

\begin{equation}
\partial_{A_t} \partial_{B_s} E_\mathrm{nuc} =
- 3 Z_{AB} r_{AB}^{-5} V_{ABt} V_{ABs}
+ 3 Z_{AM} r_{AM}^{-5} V_{AMt} V_{AMs}
+ 3 Z_{AB} r_{AB}^{-3} - Z_{AM} r_{AM}^{-3}
\end{equation}

在程序实现上，我们还会作一些初步的简化，譬如定义

* `nuc_5` : $Z_{AB} r_{AB}^{-5} V_{ABt} V_{ABs}$
* `nuc_3` : $Z_{AB} r_{AB}^{-3}$
* `mask_atm` : $\delta_{AB}$
* `mask_3D` : $\delta_{ts}$

In [ ]:
nuc_5 = 3 * np.einsum("AB, AB, ABt, ABs -> ABts", nuc_Z, nuc_rinv ** 5, nuc_V, nuc_V)
nuc_3 = np.einsum("AB, AB -> AB", nuc_Z, nuc_rinv ** 3)
mask_atm = np.eye(natm)[:, :, None, None]
mask_3D = np.eye(3)[None, None, :, :]

辅以求和与进阶的右值 Boardcasting (这里也许不太适合使用进阶的左值 Indexing 赋值)，就可以在单行完成稍复杂的指标操作．这种做法未必是写代码上友好的——针对这个任务，写代码时往往会倾向于使用循环；但在观察代码上，不带循环的、单行的代码会让代码到公式的联系更为紧密．

In [ ]:
hess_nuc = np.zeros((natm, natm, 3, 3))
hess_nuc -= nuc_5                                                        # ABts
hess_nuc += nuc_5.sum(axis=1) [:, None, :, :]       * mask_atm           # ABts -> AAts
hess_nuc += nuc_3             [:, :, None, None]    * mask_3D            # AB**
hess_nuc -= nuc_3.sum(axis=1) [:, None, None, None] * mask_atm * mask_3D # AB** -> AA**

我们已经最终求得核排斥能的二阶梯度．该张量可以与 PySCF 生成函数进行比对：

In [ ]:
np.allclose(hess_nuc, scf_hess.hess_nuc())

至此，RHF 的二阶梯度的所有贡献项已经完成：

In [ ]:
np.allclose(hess_noU + hess_withU + hess_nuc, hess_RHF)

## 代码总结

由于二阶原子核梯度的代码长度较长，我们 [另起一个文件](include/my_rhf_hess.py) 总结上面的代码．总结的原则是只使用 NumPy 与 pyscf.lib 所定义的数学与张量计算函数、pyscf.scf 所给出的基本矩阵、pyscf.scf.cphf 所进行的 CP-HF 过程以及 pyscf.gto 给出的原子轨道积分张量，并且使用函数式编程．下面的代码只是表明，我们所总结的代码是正确的：

In [ ]:
from include import my_rhf_hess
np.allclose(
    my_rhf_hess.my_hess_elec(scf_eng) + my_rhf_hess.my_hess_nuc(scf_eng),
    hess_RHF
)

我们所总结的代码的效率尽管不高于 PySCF 的代码，但也并不差到数量级级别：

In [ ]:
%%timeit -r 3 -n 3
my_rhf_hess.my_hess_elec(scf_eng) + my_rhf_hess.my_hess_nuc(scf_eng)

In [ ]:
%%timeit -r 3 -n 3
scf_hess.kernel()